In [1]:
from src.preprocessing import *
from src.metrics import *
from src.visualisation import *
from src.NetworkGraphs import *

import networkx as nx
from IPython.display import display

In [2]:
Crypto_networkGraphs = NetworkGraphs('../datasets/Dune_Eth_transaction.csv', type="CRYPTO")

In [2]:
networkGraphs = NetworkGraphs('../datasets/Railway.csv', type="RAILWAY", spatial =True)

Converting 0.644444444 to 927
Converting 0.644444444 to 927
Converting 0.695138889 to 1001
Converting 0.693055556 to 998
Converting 0.735416667 to 1059
Converting 0.733333333 to 1055
Converting 0.754166667 to 1086
Converting 0.752777778 to 1084
Converting 0.763888889 to 1100
Converting 0.7625 to 1097
Converting 0.776388889 to 1118
Converting 0.775 to 1116
Converting 0.788194444 to 1134
Converting 0.786805556 to 1133
Converting 0.815972222 to 1174
Converting 0.809722222 to 1165
Converting 0.828472222 to 1192
Converting 0.826388889 to 1190
Converting 0.850694444 to 1224
Converting 0.850694444 to 1224
Converting 0.724305556 to 1043
Converting 0.724305556 to 1043
Converting 0.743055556 to 1070
Converting 0.740972222 to 1066
Converting 0.753472222 to 1084
Converting 0.752083333 to 1082
Converting 0.760416667 to 1095
Converting 0.760416667 to 1095
Converting 0.767361111 to 1104
Converting 0.767361111 to 1104
Converting 0.775694444 to 1116
Converting 0.775694444 to 1116
Converting 0.793055556

# Graph Statistics

In [ ]:
global_metrics = compute_global_metrics(networkGraphs)
global_metrics

In [3]:
directed_node_metrics = compute_node_metrics(networkGraphs, directed=True)
directed_node_metrics

,Node,Degree Centrality,Eigenvector Centrality,Closeness Centrality,Betweeness Centrality,Load Centrality
0,692,0.008094,1.613618e-02,0.146138,0.025283,0.025286
1,1351,0.005519,4.355226e-03,0.136173,0.015457,0.014929
2,698,0.002208,6.230837e-04,0.119821,0.000544,0.000544
3,944,0.003679,2.012309e-03,0.127892,0.000967,0.001422
4,232,0.005151,6.061874e-04,0.120029,0.007658,0.007663
...,...,...,...,...,...,...
2714,2768,0.001472,1.448486e-07,0.086094,0.002926,0.002926
2715,2679,0.001472,1.301788e-08,0.079258,0.002195,0.002195
2716,2657,0.001472,1.169981e-09,0.073423,0.001464,0.001464
2717,2682,0.001472,1.051485e-10,0.068386,0.000732,0.000732


In [5]:
html = export_to_csv(directed_node_metrics, 'directed_node_metrics.csv')

NameError: name 'HTML' is not defined

In [ ]:
undirected_node_metrics = compute_node_metrics(networkGraphs, directed=False)
undirected_node_metrics

# VISUALISATION

In [ ]:
plt_directed = plot_static_on_map(networkGraphs, 'Railway Network Directed Graph', directed=True)
plt_directed.show()

In [ ]:
plt_undirected = plot_static_on_map(networkGraphs, 'Railway Network Undirected Graph', directed=False)
plt_undirected.show()

# Spatial and Temporal Analysis

### Create frames of the graph
May take very long to create all of the frame images for 2 Days, one frame is created every 5 minute. You may change the range to create less frames and a shorter video.

In [ ]:
temporal_graphs = create_temporal_subgraph(networkGraphs,0, (3*24+12)*60, 10)

In [ ]:
%matplotlib notebook
slider, plt = plot_temporal_graphs(temporal_graphs)

### Create a video of the graph with cv2 (OpenCV)

In [ ]:
# import cv2
# import os
#
# # Path to the folder containing the image frames
# frames_folder = 'frames/'
#
# # Get the list of frame filenames in the folder
# frame_filenames = os.listdir(frames_folder)
#
# # Sort the filenames in ascending order
# frame_filenames.sort(key=lambda x: int(x[:-4]))
#
# # Read the first frame to get its dimensions
# frame = cv2.imread(frames_folder + frame_filenames[0])
# height, width, layers = frame.shape
#
# # Create a VideoWriter object to write the video
# fourcc = cv2.VideoWriter_fourcc(*'mp4v')
# video = cv2.VideoWriter('output.mp4', fourcc, 30, (width, height))
#
# # Loop through the frames and add them to the video
# i=0
# for filename in frame_filenames:
#     frame = cv2.imread(frames_folder + filename)
#     video.write(frame)
#     print(f"\r{i/2764*100:.2f}%", end="")
#     i+=1
#
# # Release the VideoWriter object and display a message
# video.release()
# print('\nVideo saved as output.mp4')

# SHORTEST PATH ANALYSIS

In [ ]:
get_shortest_path(networkGraphs, source=1136, target=1095, directed=False)